In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, classification_report
import numpy as np
from sklearn.preprocessing import LabelEncoder

# Load the dataset from Excel
data = pd.read_excel('fulldataset.xlsx')

# 1. Data Cleaning and Preprocessing
# Fill missing values
for col in data.select_dtypes(include=['number']).columns:
    data[col] = data[col].fillna(data[col].median())
for col in data.select_dtypes(include=['object']).columns:
    data[col] = data[col].fillna(data[col].mode()[0])

# Convert date columns to datetime objects and extract features
date_columns = ['Date', 'Enrolldate', 'BIRTHDATE']
for col in date_columns:
    data[col] = pd.to_datetime(data[col], errors='coerce')
    data[f'{col}_year'] = data[col].dt.year
    data[f'{col}_month'] = data[col].dt.month
    data[f'{col}_day'] = data[col].dt.day
    data.drop(col, axis=1, inplace=True)  # Drop original date column

# Define target variable and features
y = data['BRD_Total']
X = data.drop(columns=['BRD_Total'])

# 2. Feature Engineering (Identify numerical and categorical columns)
numerical_cols = X.select_dtypes(include=np.number).columns.tolist()
categorical_cols = X.select_dtypes(exclude=np.number).columns.tolist()

# 3. Preprocessing Pipeline
numerical_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())
])

categorical_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
])

preprocessor = ColumnTransformer([
    ('numerical', numerical_pipeline, numerical_cols),
    ('categorical', categorical_pipeline, categorical_cols)
], remainder='passthrough')

# 4. Model Selection (Using GradientBoostingClassifier with learning rate)
model = GradientBoostingClassifier(random_state=42)

# 5. Hyperparameter Tuning (GridSearchCV)
param_grid = {
    'classifier__n_estimators': [100, 200, 300],
    'classifier__max_depth': [3, 5, 7],
    'classifier__learning_rate': [0.01, 0.05, 0.1]  # Learning rate for GradientBoosting
}

grid_search = GridSearchCV(Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', model)
]), param_grid, cv=3, scoring='accuracy', verbose=1, n_jobs=-1)

# 6. Split Data and Train Model
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
grid_search.fit(X_train, y_train)

# 7. Evaluate the Model
best_model = grid_search.best_estimator_

# Make predictions on the test set
y_pred = best_model.predict(X_test)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Model Accuracy: {accuracy * 100:.2f}%")

# Print classification report
print(classification_report(y_test, y_pred))

# 8. Table of Results: Hyperparameter Grid, Learning Rate, Training Rate, and Accuracy
results = grid_search.cv_results_

# Collecting relevant information for the results table
results_df = pd.DataFrame({
    'n_estimators': results['param_classifier__n_estimators'],
    'max_depth': results['param_classifier__max_depth'],
    'learning_rate': results['param_classifier__learning_rate'],
    'mean_test_score': results['mean_test_score']
})

# Adding accuracy (on the best model) to the table
results_df['Accuracy'] = accuracy * 100  # Since we have only one final accuracy for this grid search

# Print the table
print("\nHyperparameter Tuning Results Table:")
print(results_df)


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, classification_report
import numpy as np
from sklearn.preprocessing import LabelEncoder

# Load the dataset
data = pd.read_csv('metadata.csv')

# 1. Data Cleaning and Preprocessing
data['status'] = data['status'].fillna('unknown')
data['BRD'] = data['status'].apply(lambda x: 0 if x == 'unknown' or x is None else (1 if x == 'removed' else 0))

data['collection_date'] = pd.to_datetime(data['collection_date'], errors='coerce')
data['year'] = data['collection_date'].dt.year
data['month'] = data['collection_date'].dt.month
data['day'] = data['collection_date'].dt.day

data['clade'] = data['clade'].fillna(data['clade'].mode()[0])
data['clade'] = data['clade'].astype(int)

# 2. Feature Engineering
y = data['BRD']
X = data.drop(columns=['sample', 'strain', 'sample_title', 'organism', 'isolate', 'host', 'collection_date', 'status', 'BRD'])

numerical_cols = X.select_dtypes(include=np.number).columns.tolist()
categorical_cols = X.select_dtypes(exclude=np.number).columns.tolist()

# 3. Preprocessing Pipeline
numerical_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())
])

categorical_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
])

preprocessor = ColumnTransformer([
    ('numerical', numerical_pipeline, numerical_cols),
    ('categorical', categorical_pipeline, categorical_cols)
], remainder='passthrough')

# 4. Model Selection (Using GradientBoostingClassifier with learning rate)
model = GradientBoostingClassifier(random_state=42)

# 5. Hyperparameter Tuning (GridSearchCV)
param_grid = {
    'classifier__n_estimators': [100, 200, 300],
    'classifier__max_depth': [None, 5, 10],
    'classifier__min_samples_split': [2, 5, 10],
    'classifier__learning_rate': [0.01, 0.05, 0.1]  # Learning rate for GradientBoosting
}

grid_search = GridSearchCV(Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', model)
]), param_grid, cv=3, scoring='accuracy', verbose=1, n_jobs=-1)

# 6. Split Data and Train Model
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
grid_search.fit(X_train, y_train)

# 7. Evaluate the Model
best_model = grid_search.best_estimator_

# Make predictions on the test set
y_pred = best_model.predict(X_test)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Model Accuracy: {accuracy * 100:.2f}%")

# Print classification report
print(classification_report(y_test, y_pred))

# 8. Table of Results: Hyperparameter Grid, Learning Rate, Training Rate, and Accuracy
results = grid_search.cv_results_

# Collecting relevant information for the results table
results_df = pd.DataFrame({
    'n_estimators': results['param_classifier__n_estimators'],
    'max_depth': results['param_classifier__max_depth'],
    'min_samples_split': results['param_classifier__min_samples_split'],
    'learning_rate': results['param_classifier__learning_rate'],
    'mean_test_score': results['mean_test_score']
})

# Adding accuracy (on the best model) to the table
results_df['Accuracy'] = accuracy * 100  # Since we have only one final accuracy for this grid search

# Print the table
print("\nHyperparameter Tuning Results Table:")
print(results_df)
